In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import torch

/home/jwjeong/anaconda3/envs/test/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_id = "facebook/opt-125m"
device = "cuda" if torch.cuda.is_available() else "cpu"
#model = AutoModelForCausalLM.from_pretrained(model_id).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
dataset = dataset.shuffle(seed=42)

In [3]:
quantize_config = BaseQuantizeConfig(
    bits=4, group_size=128, desc_act=False
)

model_quant = AutoGPTQForCausalLM.from_pretrained(model_id, quantize_config=quantize_config)

/home/jwjeong/anaconda3/envs/test/lib/python3.8/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
from utils.act_scale import get_actin, get_actout, get_actin_sparsity
#fc1_actout = get_actout(model, tokenizer, 10, 512, device, ['fc1'])
fc2_actin = get_actin_sparsity(model, dataset, tokenizer, 100, 512, device, ['fc2'])

100%|██████████| 100/100 [00:01<00:00, 88.71it/s]


In [16]:
temp = []
for key in fc2_actin.keys():
    temp.append(fc2_actin[key].unsqueeze(0))
temp = torch.concat(temp, dim=0)

In [23]:
temp.std()*100

tensor(4.2077, device='cuda:0')